# 3. Usando el archivo MNIST(mnist_train_small.csv)

In [ ]:
# importamos las lib que usaremos
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import  Counter

### (a) Realiza una reducción dimensional a dos componentes principales. De todo el conjunto de datos.

In [ ]:
data = pd.read_csv('./sample_data/mnist_train_small.csv', header=None) #Leemos el archivo especificado.
data = data.to_numpy() # Pasamos data de pandas a una matriz de numpy.
X = data[:,1:]  # Seleccionamos todas las filas y columnas a partir de la segunda columna hasta el final.
y = data[:,0]   # Seleccionamos todos los elementos de la primer columna.

In [ ]:
print("DIMENSIONALIDAD")
print(f"CSV: {data.shape}")
print(f"Características: {X.shape}")
print(f"Labels/Etiquetas: {y.shape}")

In [ ]:
'''
El argumento n_components=2 en la función PCA indica que se deben seleccionar solamente las dos componentes principales más importantes
para la visualización de los datos. En otras palabras, la función PCA(n_components=2) transforma los datos originales en un conjunto de 
datos de dos dimensiones que mantiene la mayor cantidad de información posible. Esto se logra mediante la proyección de los datos originales
en un espacio de dos dimensiones que maximiza la varianza de los datos.
'''
pca = PCA(n_components=2)

In [ ]:
'''
El resultado de pca.fit_transform(data) es una nueva matriz que representa los datos 
originales en un espacio de características con una dimensión reducida, en este caso a dos dimensiones.
'''
X_2d = pca.fit_transform(data)

In [ ]:
print("REDUCCIÓN DE DIMENSIONALIDAD")
print(f"Características reducidas: {X_2d.shape}")

### (b) Separen los datos a cuyas etiqueta correspondan el 0 y 1. De tal forma que a cada etiqueta correspondan 50 muestras. Para crear un conjunto de entrenamiento.(100 Muestras)

In [ ]:
# Obtenemos los datos correspondientes a la clase 0 y 1, así como las variables auxiliares para graficar los datos
X0 = []
X1 = []
X0_x,X0_y,X1_x,X1_y = [],[],[],[]
for c, x in zip(y,X_2d):
  if c==0:
    X0.append(x)
    X0_x.append(x[0])
    X0_y.append(x[1])
  elif c==1:
    X1.append(x)
    X1_x.append(x[0])
    X1_y.append(x[1])

In [ ]:
# Generamos una gráfica, tal que los puntos rojos pertenecen a la clase cero y los azules a la clase uno.
plt.scatter(X0_x, X0_y, c="r")
plt.scatter(X1_x, X1_y, c="b")
plt.show()

In [ ]:
#Funcion auxiliar
def getOnesZeros(n):
  return np.zeros(n).tolist()+np.ones(n).tolist()

In [ ]:
x0 = X0[0:50]
x1 = X1[0:50]
X = x0+x1
y_train = np.array([int(y) for y in getOnesZeros(50)]) # Clases correspondientes al conjunto de entrenamiento.
X_train = np.array([x.tolist() for x in X]) # Conjunto de entrenamiento.

### (c) Gráfica la malla de la clasificación y los puntos de los datos completos de ambas etiquetas. Usando K-NN.

In [ ]:
# Creamos el modelo KNN y entrenamos
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Creamos una malla de puntos para clasificar
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max), np.arange(y_min, y_max))

# Predecimos la clase para cada punto en la malla
c = np.c_[xx.ravel(), yy.ravel()]
Z = knn.predict(c)
Z = Z.reshape(xx.shape)

# Graficamos la malla de clasificación
plt.figure(figsize=(8, 6))
#plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)
plt.pcolormesh(xx, yy, Z, alpha=0.4)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k')
plt.title('Malla de Clasificación para KNN')
plt.show()

In [ ]:
# Función para calcular la distancia euclidiana
def euclidean_distance(point, centroid):
    return np.sqrt(np.sum((point - centroid)**2))

# Generamos el conjunto de prueba
x0_test = X0[50:len(X0)]
x1_test = X1[50:len(X1)]
X_test = np.array(x0_test+x1_test)
y_test = np.zeros(len(x0_test)).tolist() + np.ones(len(x1_test)).tolist()
y_test = np.array([int(y) for y in y_test])

def k_nearest_neighbors(X_train, y_train, X_test, k=3):
  #Lista para almacenar las clases asignadas
  y_pred = []
  X_test = np.array(X_test)

  #Iterar sobre cada punto de prueba
  for ind in range(X_test.shape[0]):
    #Calcular las distancias entre el punto de prueba y todos los puntos de entrenamiento.
    distancias = [euclidean_distance(X_test[ind], p_ent ) for p_ent in X_train ]
    
    #Seleccionar los K puntos más cercanos
    K_indices = np.argsort(distancias)[:k] 
    
    # Asignar la clase más frecuente entre los K vecinos más cercanos al punto de prueba
    k_nearest_classes = [y_train[idx] for idx in K_indices]
    
    #Seleccionamos la clase más comun. [(valor,numero_de_repeticiones)]
    most_common_clas = Counter(k_nearest_classes).most_common(1)
    #Asigamos a la clase más commun.
    y_pred.append(most_common_clas[0][0])
  return y_pred

In [ ]:
plt.figure(figsize=(8,8),dpi=100)
n_vec = 5

# Predecir clases para cada punto en X_space
y_space_pred = k_nearest_neighbors(X_train, y_train, X_test, k=n_vec)

# Graficar los puntos de X_space en diferentes colores según su clase asignada
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_space_pred, cmap='Accent', alpha=.3)#label='Predicted Classes'

# Graficar los puntos de entrenamiento y los puntos de prueba en diferentes colores
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='Accent', label='Training Points')
#Conjunto de prueba
plt.scatter(X_test[:, 0], X_test[:, 1],marker='x', c=y_space_pred,cmap='Accent', label='Test Points')
plt.title(f'Vecinos k{n_vec}')
plt.legend()
plt.show()

### (d) Usando `sklearn.metrics.classification_report()`, cree un informe que muestre las principales métricas de clasificación (Precision, Recall, etc). Usando como `X_test` los datos completos del conjunto de datos y como `X_train` El conjunto de datos con las 100 muestras.

In [ ]:
def imprimir_reporte(reporte):
    for k in report:

        if (k == "0" or k == "1"):
            print(f"Para la clase {k}...")
        elif (k == "accuracy"):
            print(f"De manera general, {k}: {report[k]}\n")
            continue
        else:
            print(f"{k}...")

        for k,v in report[k].items():
            print(f"\t{k}: {v}")
        print()

In [ ]:
y_pred = k_nearest_neighbors(X_train, y_train, X_test, k=3)
report = classification_report(y_test, y_pred, output_dict=True)
imprimir_reporte(report)

### (e) Discutan en equipo y den una conclusión de lo que se observa en los datos. ¿Qué puedes decir acerca de las muestras si se usaran 10 muestras por clase? ¿Son muchas o pocas?

In [ ]:
# Generamos los mismos conjuntos pertinentes, pero ahora usando 10 muestras por clase...
x0 = X0[0:10]
x1 = X1[0:10]
X = x0+x1
y_train = np.array([int(y) for y in getOnesZeros(10)])
X_train = np.array([x.tolist() for x in X])
x0_test = X0[10:len(X0)]
x1_test = X1[10:len(X1)]
X_test = np.array(x0_test+x1_test)
y_test = np.zeros(len(x0_test)).tolist() + np.ones(len(x1_test)).tolist()
y_test = np.array([int(y) for y in y_test])

# Generamos el reporte correspondiente.
y_pred = k_nearest_neighbors(X_train, y_train, X_test, k=3)
report = classification_report(y_test, y_pred, output_dict=True)
imprimir_reporte(report)

En este caso, parece que el número de muestras en el conjunto de entrenamiento no afecta en gran medida la exactitud del modelo basado en KNN. Pues se ven resultados casi idénticos con un número menor de muestras.

Podemos decir que, dado el número de clases de clasificación, la cardinalidad del conjunto de entrenamiento no afecta desmedidamente al modelo.

Por lo tanto, usar 50 muestras por clases se considera como muchas muestras, mientras que usar 10 se considera suficiente.